In [1]:
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [2]:
with open('./processed_song.json', 'r', encoding='UTF8') as f:
    data = json.load(f)
df = pd.json_normalize(data)
df = df[df['count'] > 0]

In [3]:
# 태그들의 리스트, K-means clustering을 위한 input 값
content = []
for idx, row in df.iterrows():
    string = ' '.join(row.tags)
    df.at[idx, 'tag'] = string
    content.append(string)

In [4]:
print(len(content))

74259


In [12]:
n_clusters = 10
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(content)

kmeans = KMeans(n_clusters=n_clusters).fit(X)

labels = kmeans.labels_
centers = kmeans.cluster_centers_
df['labels'] = labels

In [13]:
df.head(5)

,id,song_name,artist_name_basket,tags,count,tag,labels
0,144663,밤편지,[아이유],"[감성, 발라드, 새벽, 밤, 인디, 비오는날, 슬픔, 새벽감성, 여름밤, 감성발라...",2175,감성 발라드 새벽 밤 인디 비오는날 슬픔 새벽감성 여름밤 감성발라드 비 혼자 잠들기...,3
1,116573,안아줘,[정준일],"[발라드, 이별, 새벽, 감성, 밤, 슬픔, 힐링, 기분전환, 가을, 카페, 비오는...",2121,발라드 이별 새벽 감성 밤 슬픔 힐링 기분전환 가을 카페 비오는날 외로움 쌀쌀한 쓸...,3
2,357367,비,[폴킴],"[감성, 발라드, 비오는날, 이별, 새벽, 사랑, 비, 휴식, 밤, 힐링, 센치, ...",1981,감성 발라드 비오는날 이별 새벽 사랑 비 휴식 밤 힐링 센치 혼자 장마 감성발라드 ...,3
3,366786,가끔 미치도록 네가 안고 싶어질 때가 있어,[가을방학],"[새벽, 인디, 밤, 감성, 발라드, 힐링, 추억, 기분전환, 카페, 비오는날, 인...",1919,새벽 인디 밤 감성 발라드 힐링 추억 기분전환 카페 비오는날 인디음악 락 새벽감성 ...,3
4,133143,"그대와 나, 설레임 (Feat. 소울맨)",[어쿠스틱 콜라보],"[기분전환, 힐링, 카페, 발라드, 감성, 새벽, 밤, 기분좋은, 인디음악, 설레임...",1633,기분전환 힐링 카페 발라드 감성 새벽 밤 기분좋은 인디음악 설레임 주말 분위기 편안...,3


In [14]:
def get_cluster_details(cluster_model, cluster_data, feature_names,
                       cluster_num, top_n_features=10):
    cluster_details = {}
    center_feature_idx = cluster_model.cluster_centers_.argsort()[:,::-1]
    
    for cluster_num in range(cluster_num):
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        
        top_ftr_idx = center_feature_idx[cluster_num, :top_n_features]
        top_ftr = [feature_names[idx] for idx in top_ftr_idx]
        top_ftr_val = cluster_model.cluster_centers_[cluster_num, top_ftr_idx].tolist()
        
        cluster_details[cluster_num]['top_features'] = top_ftr
        cluster_details[cluster_num]['top_featrues_value'] = top_ftr_val
        filenames = cluster_data[cluster_data['labels']==cluster_num]['tag']
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['filenames'] = filenames
    
    return cluster_details
feature_names = vectorizer.get_feature_names()
details = get_cluster_details(cluster_model=kmeans, cluster_data=df, feature_names=feature_names, cluster_num=n_clusters)

/Users/juhyunlee/miniconda3/envs/MLvenv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [15]:
len(details)

10

In [16]:
for key, value in details.items():
    print(key, value['top_features'])

0 ['힙합', '인디', '알앤비', '기분전환', '감성', '드라이브', '발라드', '매장음악', '소울', '휴식']
1 ['댄스', '드라이브', '신나는', '운동', '아이돌', '기분전환', '스트레스', '여행', '여름', '걸그룹']
2 ['힙합', '발라드', '알앤비', '기분전환', '소울', '새벽', '드라이브', '스트레스', '감성', '인디']
3 ['잔잔한', '새벽', '감성', '인디', '카페', '휴식', '발라드', '가을', '이별', '힐링']
4 ['기분전환', '스트레스', '드라이브', '휴식', '여름', '매장음악', '사랑', '새벽', '까페', '운동']
5 ['힐링', '휴식', '기분전환', '잔잔한', '새벽', '까페', '설렘', '사랑', '발라드', '산책']
6 ['인디', '잔잔한', '가요', '새벽', '사랑', '기분전환', '감성', '겨울', '인디음악', '가을']
7 ['설렘', '사랑', '기분전환', '달달', '까페', '연애', '휴식', '인디', '힐링', '고백']
8 ['회상', '추억', '슬픔', '이별', '새벽', '발라드', '사랑', '잔잔한', '힐링', '휴식']
9 ['발라드', '인디', '댄스', '사랑', '잔잔한', '가요', '감성', '새벽', '기분전환', '매장음악']


In [11]:
for i in range(0, 20):
    print(len(df[df['labels'] == i]))

346
9678
4530
1056
935
4950
1777
1676
1133
5977
1106
685
2479
787
1024
925
25620
7996
776
803
